# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.54s/it]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [2024-09-02 11:03:41][DEBUG]: == Working Agent: Tech Job Researcher
 [2024-09-02 11:03:41][INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [2024-09-02 11:03:41][DEBUG]: == Working Agent: Personal Profiler for Engineers
 [2024-09-02 11:03:41][INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiative

- Dislplay the generated `tailored_resume.md` file.

In [18]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams  
- **Email:** noah.williams@example.dev  
- **Phone:** +44 11 111 11111  
- **Location:** Colombia  

## Profile  
Dynamic Full Stack Engineer with 18 years of experience in software development, focusing on building scalable applications and robust software solutions. Proven expertise in both front-end and back-end technologies, including HTML, CSS, JavaScript, React, Angular, Python, and Ruby. Adept at collaborating with cross-functional teams and managing the full software development lifecycle, from ideation to deployment. Strong background in data protection, security, and performance optimization, with a commitment to delivering high-quality software that meets user needs and drives business growth.

## Skills  
- **Programming Languages:** Ruby, Python, JavaScript, TypeScript, Elixir  
- **Frameworks:** React, Angular, Ruby on Rails, Node.js  
- **Databases:** MongoDB, MySQL  
- **Technologies:** AI, Machine Learning, IoT, Data Analytics  
- **APIs:** RESTful services, API development and integration  
- **DevOps:** CI/CD, Git, Agile methodologies  
- **Soft Skills:** Excellent communication, project management, leadership, problem-solving  

## Work History  

### DataKernel: Director of Software Engineering (remote) — 2022 - Present  
- Transformed engineering division into a key revenue-generating unit, expanding the customer base significantly.  
- Spearheaded the integration of scalable AI solutions and vector databases, enhancing product capabilities.  
- Defined strategic initiatives to adopt cutting-edge technologies that set new industry benchmarks.  

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022  
- Directed engineering strategy and operations in collaboration with C-level executives to shape technological direction.  
- Managed diverse teams across North America and Europe, fostering transparency and high performance.  
- Led recruitment and mentoring efforts, cultivating a culture of continuous improvement.  

### InnovPet: Founder & CEO (remote) — 2019 - 2022  
- Founded a startup focused on IoT solutions for pet care, creating innovative products that improved pet safety.  
- Oversaw product development from concept to execution, ensuring successful market entry.  
- Built the initial product using MongoDB and implemented agile methodologies to streamline development.  

### EliteDevs: Engineering Manager (remote) — 2018 - 2019  
- Formulated strategic plans to enhance inter-departmental collaboration and project outcomes.  
- Managed multiple engineering teams, balancing productivity with innovation.  
- Actively contributed to product development using Ruby on Rails and React.  

### PrintPack: Engineering Manager (remote) — 2016 - 2018  
- Led a high-performance engineering team, achieving a 500% increase in revenue within two years.  
- Integrated data analytics into decision-making processes, enhancing customer behavior insights.  

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016  
- Developed and optimized central APIs, improving functionality and performance for a large user base.  
- Implemented caching strategies that reduced response times significantly.  

### BetCraft: CTO — 2013 - 2015  
- Guided technology strategy post-Series A funding, enhancing platform performance and market reach.  
- Developed initial products using React and Angular, significantly improving user engagement.  

## Education  
- **MBA in Information Technology**  
  London Business School  

- **Advanced Leadership Techniques**  
  University of London - Certification  

- **Data Science Specialization**  
  Coursera (Johns Hopkins University) - Certification  

- **B.Sc. in Computer Science**  
  University of Edinburgh  

## Interests  
Passionate about leveraging technology for social good, particularly in AI and IoT. Enjoy mentoring emerging engineers and advocating for inclusive work environments that foster collaboration and growth.

## Communication Style  
Noah adopts a transparent and collaborative communication style that promotes open dialogue and encourages team members to share ideas and feedback, ensuring alignment and fostering team dynamics.

This resume effectively highlights Noah's qualifications and experiences, aligning closely with the Full Stack Engineer position at AI Fund, showcasing relevant skills and achievements that directly respond to the job requirements.

- Dislplay the generated `interview_materials.md` file.

In [19]:
display(Markdown("./interview_materials.md"))

# Interview Preparation Document for Noah Williams - Full Stack Engineer Position at AI Fund

## Potential Interview Questions and Talking Points

### General Background and Experience
1. **Can you describe your journey in the technology industry?**
   - Highlight your 18 years of experience and the evolution from a Senior Software Engineer to a Director of Software Engineering. Emphasize your ability to lead both remote and in-office teams.

2. **What inspired you to pursue a career in software engineering?**
   - Discuss your passion for technology and how it has been a driving force throughout your career.

3. **What do you consider your biggest achievement in your career so far?**
   - Mention the transformation of DataKernel's engineering division into a revenue-generating unit and the expansion of the customer base.

### Technical Skills and Proficiencies
4. **Can you elaborate on your experience with front-end and back-end technologies?**
   - Discuss your proficiency in HTML, CSS, JavaScript, React, Angular, Python, Ruby, and how you have utilized these technologies in your projects.

5. **Describe a project where you developed a scalable application. What challenges did you face?**
   - Talk about your work at DataKernel or InnovPet, focusing on the integration of AI technologies and scalable vector databases.

6. **How do you ensure the databases you manage are efficient and secure?**
   - Highlight your experience with MongoDB and MySQL, and discuss your approach to creating security and data protection settings.

7. **What is your experience with API development and integration?**
   - Share your experience in developing RESTful services and the central API you optimized at DriveAI.

### Software Development Lifecycle
8. **How do you approach the software development lifecycle?**
   - Discuss your experience managing projects from conception to deployment, focusing on Agile methodologies and CI/CD practices.

9. **Can you provide an example of how you've tested software for responsiveness and efficiency?**
   - Share specific examples, such as the caching strategies you implemented at DriveAI that improved system performance.

10. **How do you troubleshoot and debug software issues? Can you give an example of a complex problem you solved?**
    - Talk about a specific instance where your problem-solving skills were crucial in resolving a significant software issue.

### Team Management and Collaboration
11. **How do you foster collaboration among cross-functional teams?**
    - Emphasize your leadership style focused on transparency and mutual respect, as demonstrated in your roles at DataKernel and EliteDevs.

12. **Describe your experience in mentoring and developing team members.**
    - Discuss your initiatives in recruitment, mentoring, and creating a culture of continuous improvement.

13. **How do you handle team dynamics in a remote work environment?**
    - Talk about your strategies for maintaining high-performance team dynamics across multiple time zones.

### Leadership and Strategic Vision
14. **What leadership qualities do you believe are most important in a software engineering role?**
    - Reflect on accountability, grit, and ownership orientation, and how these qualities have shaped your leadership style.

15. **How do you balance innovation with productivity within your teams?**
    - Share your strategies for managing multiple engineering teams and ensuring alignment with long-term goals.

16. **What role do you think AI/ML will play in the future of software development?**
    - Discuss your insights on the integration of AI technologies in software solutions, based on your experience at DataKernel.

### Cultural Fit and Company Values
17. **Why are you interested in working with AI Fund?**
    - Share your passion for leveraging technology for social good and your interest in working with diverse teams.

18. **How do you align your personal values with those of the company you're working for?**
    - Discuss your commitment to diversity, inclusion, and creating collaborative work environments.

### Questions for the Interviewers
19. **What are the current challenges the team is facing that I could help address?**
20. **Can you describe the team dynamics and culture at AI Fund?**
21. **What opportunities for professional development does AI Fund offer?**

### Closing Statement
- **Thank the interviewers for their time and express your enthusiasm for the opportunity to contribute to AI Fund's mission.**
- **Reiterate your alignment with the role and how your extensive experience can add value to the team.**

By preparing these questions and talking points, Noah will be well-equipped to highlight his qualifications and experiences, demonstrating a strong match for the Full Stack Engineer position at AI Fund.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)